In [35]:
import os
import torch
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np
import torch.nn as nn
import torchvision.models as models
from torch.autograd import Variable
import pandas as pd


data_dir = os.path.join("./", "TrainTest")

In [36]:
resnet18 = models.resnet18(pretrained=True)
modules=list(resnet18.children())[:-1]
resnet18=nn.Sequential(*modules)
for p in resnet18.parameters():
    p.requires_grad = False

In [50]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
dataset = datasets.ImageFolder(data_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True, num_workers=0)

image_features = np.zeros((1, 513))

#print(image_features)
for data in dataloader:
    image, label = data
    encoding = resnet18(image) 
    encoding_np = encoding.numpy()
    encoding_np = encoding_np.reshape(1, 512)
    encoding_np = np.append(encoding_np, label, axis=1)
  

    df = pd.DataFrame(encoding_np)
    print(df)
    #print(image_features)
    
    
    #image_features = np.append(image_features, encoding_np, axis=0)
    #df = pd.DataFrame(image_features)
    #print(df)
    #print(image_features)
    break
     

    #print(image.size())
    #encoding = model(image) 
    #im_np = image.numpy()
    #print(im.size())
    # #plt.imshow(im_np)
    #plt.imshow(torch.permute(im, (1, 2, 0)))
    #plt.show()
    #break

# drop first row of zeros
image_features = image_features.iloc[1: , :]

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)